## Estilização da Saída do Markdown

In [1]:
%%HTML
<style>
  @import url('https://fonts.googleapis.com/css2?family=JetBrains+Mono&display=swap');

  body {
    font-family: 'JetBrains Mono', -apple-system, BlinkMacSystemFont, 'Segoe WPC', 'Segoe UI', system-ui, 'Ubuntu', 'Droid Sans', sans-serif;
    line-height: 1.67;
  }
</style>


# Simulação 1 de um Conversor Buck com CPL

## 1. Importação de Bibliotecas e Implementação de Funções Utilitárias

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import control as ct
import control.optimal as opt
from control.matlab import *
import cvxpy as cp
import math
ct.use_fbs_defaults()

In [3]:
def show_matrix(name, matrix, decimal_places=2):
  """
  Apresenta uma matriz com a quantidade de casas decimais desejadas.

  Parâmetros:
  ---
  - matrix: numpy.ndarray, a matriz a ser apresentada.
  - casas_decimais: int, o número de casas decimais desejadas (padrão é 2).
  """
  pattern = "{:." + str(decimal_places) + "e}"

  def format_elem(elem):
    return pattern.format(elem)

  width = [max(map(len, map(format_elem, coluna))) for coluna in matrix.T]

  print(name, "=")

  nspaces = sum(width) + 2 * matrix.shape[1]

  print("    ┌" + " " * nspaces + "┐")
  for line in matrix:
    formatted_line = "  ".join(format_elem(e).rjust(largura)
                               for e, largura in zip(line, width))
    print("    │ " + formatted_line + " │")
  print("    └" + " " * nspaces + "┘")
  print()

In [4]:
def set_axs(axs, x, y, label, x_label, y_label, title):
  axs.plot(x, y, linestyle='-', color='black', label=label, linewidth=1.)
  axs.set_xlabel(x_label)
  axs.set_ylabel(y_label)
  axs.set_title(title)
  # axs.legend()
  axs.grid(linestyle='--')
  axs.tick_params(axis='both', direction='in', length=4, width=1,
                  colors='black', grid_alpha=1., top=True, right=True)

### 3.4 Implementação do Modelo Não-linear

In [5]:
def update_buck_nonlinear(t, x, u, params):
  # Parâmetros do sistema
  V_IN = params.get('Vin')  # Tensão de Entrada
  RL = params.get('rL')     # Resistência (indutor)
  RC = params.get('rC')     # Resistência (capacitor)
  L = params.get('L')       # Indutância
  C = params.get('C')       # Capacitância

  # Entradas do sistema: Duty Cycle e Potência da CPL
  D, P_CPL = u

  # Estados do sistema: corrente do indutor e tensão do capacitor
  IL, VC = x

  # Atualização da corrente do indutor
  diL = (V_IN / L) * D - (RL / L) * IL - VC / L

  # Atualização da tensão do capacitor
  dvC = IL / C - VC / (C * RC) - P_CPL / (C * VC)

  dx = np.array([diL, dvC])
  return dx

# Definição da saída do sistema


def output_buck_nonlinear(t, x, u, params):
  return x[0:2]


# Definição do conversor cc-cc buck nao-linear
buck_nonlinear = ct.ss(
    update_buck_nonlinear,
    output_buck_nonlinear,
    name='buck_nonlinear',
    inputs=('d', 'P_cpl'),
    outputs=('iL', 'vC'),
    states=('iL', 'vC')
)

## 4. Modelo Não-linear Trasladado

In [6]:
def update_translated_nonlinear(t, x, u, params):
  """
  Atualiza os estados do sistema não linear transladado.

  Parâmetros:
  - t: tempo
  - x: estados do sistema
  - u: entradas do sistema
  - params: parâmetros do sistema

  Retorna:
  - dx: derivadas dos estados do sistema
  """
  # Definição dos parâmetros do sistema
  V_IN = params.get('Vin')
  RL = params.get('rL')
  RC = params.get('rC')
  L = params.get('L')
  C = params.get('C')
  OP = params.get('op')

  # Definição das entradas do sistema
  δD, δP_CPL = u

  # Definição dos estados do sistema
  δIL, δVC = x

  # Atualização
  DELTA_IL_DOT = - (RL / L) * δIL - δVC / L + (V_IN / L) * δD
  DELTA_VC_DOT = - δVC / (C * RC) + δIL / C + \
      (OP['Pcpl'] * δVC - OP['vC'] * δP_CPL) / \
      (C * OP['vC'] * (OP['vC'] + δVC))

  X_DOT = np.array([DELTA_IL_DOT, DELTA_VC_DOT])

  return X_DOT


def system_output(t, x, u, params):
  """
  Calcula a saída do sistema.

  Parâmetros:
  - t: tempo
  - x: estados do sistema
  - u: entradas do sistema
  - params: parâmetros do sistema

  Retorna:
  - y: saída do sistema
  """

  return x[:2]


translated_nonlinear_system = ct.NonlinearIOSystem(
    update_translated_nonlinear, system_output,
    name='translated_nonlinear_system',
    inputs=('δd', 'δPcpl'),
    outputs=('δiL', 'δvC'),
    states=('δiL', 'δvC')
)

## 5. Modelo do Sistema Linearizado

### 5.1 Descrição

O sistema linearizado, derivado a partir do sistema não linear, obtido na <a href="#section-3" style="color: #ffffff67; text-decoration: none;">Seção 3</a>, em torno dos pontos de operação, é representado por um conjunto de equações diferenciais lineares. Essas equações descrevem as variações no tempo das grandezas $\delta i_L$ e $\delta v_C$, que representam as alterações na corrente do indutor e na tensão do capacitor, respectivamente.

Baseado nisto, o sistema linearizado terá a forma:

$$
  \begin{cases}
    \frac{d}{dt} (\delta i_L) &= k_1 \cdot \delta i_L + k_2 \cdot \delta v_C + k_3 \cdot \delta d + k_4 \cdot \delta P_{CPL} \\ \\
    \frac{d}{dt} (\delta v_C) &= k_5 \cdot \delta i_L + k_6 \cdot \delta v_C + k_7 \cdot \delta d + k_8 \cdot \delta P_{CPL}
  \end{cases}
$$

Considerando as seguintes funções a seguir,

$$f(P) = \frac{d}{dt}i_L, \space\space\space\space  g(P) = \frac{d}{dt}v_C$$

onde $P = (i_L, \space v_C, \space d, \space P_{CPL})$, podemos obter os coeficientes $k$ com as seguintes derivadas parciais:
  
$$
k_1 = \dfrac{\partial f}{\partial i_L} \vert_{P = P_O}, \space \space
k_2 = \dfrac{\partial f}{\partial v_C} \vert_{P = P_O}, \space \space
k_3 = \dfrac{\partial f}{\partial d} \vert_{P = P_O}, \space \space
k_4 = \dfrac{\partial f}{\partial P_{CPL}} \vert_{P = P_O}
$$

$$
k_5 = \dfrac{\partial g}{\partial i_L} \vert_{P = P_O}, \space \space
k_6 = \dfrac{\partial g}{\partial v_C} \vert_{P = P_O}, \space \space
k_7 = \dfrac{\partial g}{\partial d} \vert_{P = P_O}, \space \space
k_8 = \dfrac{\partial g}{\partial P_{CPL}} \vert_{P = P_O}
$$

<br>

onde $P_O = (\overline{i_L}, \space \overline{v_C}, \space \overline{d}, \space \overline{P_{CPL}})$, é o ponto de operação definido na <a href="#section-4" style="color: #ffffff67; text-decoration: none;">Seção 4</a>. Desta forma, os termos são:

<br>

$$
k_1 = - \frac{R_L}{L}, \space\space\space\space
k_2 = - \frac{1}{L}, \space\space\space\space
k_3 = \frac{V_{in}}{L} , \space\space\space\space
k_4 = 0
$$

$$
k_5 = \frac{1}{C}, \space\space\space\space
k_6 = \frac{1}{C}\left(\frac{\overline{P_{CPL}}}{{{\overline{v_{C}}}^2}} - \frac{1}{R_C}\right), \space\space\space\space
k_7 = 0, \space\space\space\space
k_8 = - \frac{1}{C \overline{v_C}}
$$

<br>

Portanto, o sistema linearizado é:

<br>

$$
\begin{equation}
\tag{5.1}
  \begin{cases}
    \displaystyle \frac{d}{dt} (\delta i_L) &= \displaystyle - \frac{R_L}{L} \delta i_L - \frac{1}{L} \delta v_C + \frac{V_{in}}{L} \delta d 
    \\ \\
    \displaystyle  \frac{d}{dt} (\delta v_C) &= \displaystyle \frac{1}{C} \delta i_L + \frac{1}{C}\left(\frac{\overline{P_{CPL}}}{{\overline{{v_{C}}}^2}} - \frac{1}{R_C}\right) \delta v_C - \frac{1}{C \overline{v_C}} \delta P_{CPL}
  \end{cases}
\end{equation}
$$

ou,

$$
  \begin{bmatrix}
    \dot{\delta i_L} \\ \dot{\delta v_C}
  \end{bmatrix}
  =
  \begin{bmatrix}
    \displaystyle -\frac{R_L}{L} & \displaystyle -\frac{1}{L} \\ \\
    \displaystyle \frac{1}{C} & \displaystyle \frac{1}{C}\left(\frac{\overline{P_{CPL}}}{{{\overline{v_{C}}}^2}} - \frac{1}{R_C}\right)
  \end{bmatrix}
  \begin{bmatrix}
    \delta i_L \\ \delta v_C
  \end{bmatrix}
  +
  \begin{bmatrix}
    \displaystyle{\frac{V_{in}}{L}} & 0 \\ \\ 
    0 & \displaystyle{-\frac{1}{C\overline{v_C}}}
  \end{bmatrix}
  \begin{bmatrix}
    \delta d \\ \delta P_{CPL}
  \end{bmatrix}
$$

<br>

### 5.2 Implementação do Sistema Linearizado

O código a seguir implementa o modelo linearizado em torno de um ponto de operação. Os parâmetros, entradas e estados do sistema são definidos.

In [7]:
# Obtenção dos valores no ponto de operação (OP)
OP = params['op']

# Elementos da matriz de estados
A11 = - (params['rL'] / params['L'])
A12 = - (1. / params['L'])
A21 = 1. / params['C']
A22 = (1. / params['C']) * (OP['Pcpl'] /
                            (OP['vC'] * OP['vC']) - 1. / params['rC'])

# Elementos da matriz de entrada
B11 = params['Vin'] / params['L']
B12 = 0.
B21 = 0.
B22 = - 1.0 / (params['C'] * OP['vC'])

# Matriz de estados: iL e vC
A = [[A11, A12], [A21, A22]]

# Matriz de entradas: d e P_cpl
B = [[B11, B12], [B21, B22]]

# Matriz de saída: iL e vC
C = [[1., 0], [0., 1]]

# Matriz de alimentação: nula
D = [[0., 0.], [0., 0.]]

buck_linearized = ct.ss2io(
    ss(A, B, C, D),
    name='buck_linearized',
    inputs=('δd', 'δPcpl'),
    outputs=('δiL', 'δvC'),
    states=('δiL', 'δvC')
)

NameError: name 'params' is not defined

## 6. Simulação dos Modelos Não-lineares

In [ ]:
def gen_u(timepts, values, D):
  U = np.zeros((2, len(timepts)))
  U[0, :] = D

  for i, t in enumerate(timepts):
    for j in range(len(values)-1):
      if values[j][0] <= t < values[j+1][0]:
        U[1, i] = values[j][1]
        break
    else:
      U[1, i] = values[-1][1]
  return U

In [ ]:
# Parâmetros do Circuito

params = {'Vin': 48, 'rL': 0.1, 'rC': 10,
          'L': 1e-3, 'C': 2.2e-3, 'op': {'Pcpl': 50, 'vC': 28}}

# Cálculo da Corrente e Duty Cycle de Operação
op = params['op']
IL_OP = (op['vC'] / params['rC']) + op['Pcpl'] / op['vC']
D_OP = (params['rL'] * IL_OP) / params['Vin'] + op['vC'] / params['Vin']

params['op']['iL'] = IL_OP
params['op']['d'] = D_OP

# Ponto de operação de cada entrada e estado do sistema
U_OP = np.array([params['op']['d'], params['op']['Pcpl']])
X_OP = np.array([params['op']['iL'], params['op']['vC']])

step = 1e-6
timepts = np.arange(0, .1 + step, step)

# Entradas do Sistema
D = params['op']['d']
P_CPL = params['op']['Pcpl']

values = [
    (0., 50)
]

U = gen_u(timepts, values, D)

# Estados Iniciais do Sistema
IL_INIT = 1.5 * params['op']['iL']
VC_INIT = 0.13 * params['op']['vC']

# IL_INIT = .95 * params['op']['iL']
# VC_INIT = .95 * params['op']['vC']
X0 = np.array([IL_INIT, VC_INIT])

δU = U - U_OP[:, np.newaxis]
δX0 = X0 - X_OP

print(δU)
print(δX0)

print('Parâmetros: ')
params

In [ ]:
linewidth = 2.5

plt.figure(figsize=(6, 3.5))
plt.plot(timepts, U[1],
         linestyle='-', color='black', linewidth=linewidth, alpha=1)
# line3 = plt.axhline(y=params['op']['iL'], color='black',
#                        linestyle='-.', label='OP', linewidth=0.5)
plt.xlabel('Tempo (s)', fontsize=18)
plt.ylabel('$P_{cpl}$ (W)', fontsize=18)
plt.grid(linestyle='--')
plt.title('Potência da $P_{cpl}(t)$', fontsize=20)
plt.tick_params(axis='both', direction='in', length=4, width=1,
                colors='black', top=True, right=True, labelsize=16)

# plt.tight_layout()
plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/buck/sim2_power_cpl_variation.eps', format='eps', bbox_inches='tight')

plt.show()

#### 5.2 Resposta do Sistema Não Linear

In [ ]:
# Obtenção da saída
t, y_nonlinear = ct.input_output_response(
    sys=buck_nonlinear, T=timepts,
    U=U.tolist(),
    X0=X0.tolist(),
    params=params,
)

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico 1 - Corrente do Indutor
axs[0].plot(t, y_nonlinear[0], linestyle='-', color='b', label='$i_L$')
axs[0].set_xlabel('Tempo(s)')
axs[0].set_ylabel('Valores de $i_L\,(A)$')
axs[0].set_title('Corrente do Indutor')
axs[0].legend()
axs[0].grid(True)

# Gráfico 2 - Tensão do Capacitor
axs[1].plot(t, y_nonlinear[1], linestyle='-', color='r', label='$v_C$')
axs[1].set_xlabel('Tempo(s)')
axs[1].set_ylabel('Valores de $v_C\,(V)$')
axs[1].set_title('Tensão do Capacitor')
axs[1].legend()
axs[1].grid(True)

fig.suptitle('Sistema Dinâmico Não-linear', fontsize=16)

plt.tight_layout()
plt.show()

### 5.3 Resposta do Sistema Não-linear Transladado

In [ ]:
# Obtenção da Saída do Sistema Não-Linear Trasladado
t, y_translated_nonlinear = ct.input_output_response(
    sys=translated_nonlinear_system, T=timepts,
    U=δU.tolist(),
    X0=δX0.tolist(),
    params=params,
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': 1e-4}
)

# Criação e apresentação do gráfico
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico 1 - Corrente do Indutor
axs[0].plot(
    t,
    y_translated_nonlinear[0] + params['op']['iL'],
    linestyle='-', color='b', label='$\delta i_L$'
)

axs[0].set_xlabel('Tempo(s)')
axs[0].set_ylabel('Valores de $\delta i_L\,(A)$')
axs[0].set_title('Corrente do Indutor')
axs[0].legend()
axs[0].grid(True)

# Gráfico 2 - Tensão do Capacitor
axs[1].plot(
    t,
    y_translated_nonlinear[1] + params['op']['vC'],
    linestyle='-', color='r', label='$\delta v_C$'
)

axs[1].set_xlabel('Tempo(s)')
axs[1].set_ylabel('Valores de $\delta v_C\,(V)$')
axs[1].set_title('Tensão do Capacitor')
axs[1].legend()
axs[1].grid(True)

fig.suptitle('Sistema Dinâmico Não-linear Transladado', fontsize=16)

plt.tight_layout()
plt.show()

## 6. Modelo do Sistema Linearizado

### 6.2 Implementação do Sistema Linearizado

In [ ]:
# Obtenção dos valores no ponto de operação (OP)
OP = params['op']

# Elementos da matriz de estados
A11 = - (params['rL'] / params['L'])
A12 = - (1. / params['L'])
A21 = 1. / params['C']
A22 = (1. / params['C']) * (OP['Pcpl'] /
                            (OP['vC'] * OP['vC']) - 1. / params['rC'])

# Elementos da matriz de entrada
B11 = params['Vin'] / params['L']
B12 = 0.
B21 = 0.
B22 = - 1.0 / (params['C'] * OP['vC'])

# Matriz de estados: iL e vC
A = [[A11, A12], [A21, A22]]

# Matriz de entradas: d e P_cpl
B = [[B11, B12], [B21, B22]]

# Matriz de saída: iL e vC
C = [[1., 0], [0., 1]]

# Matriz de alimentação: nula
D = [[0., 0.], [0., 0.]]

buck_linearized = ct.ss2io(
    ss(A, B, C, D),
    name='buck_linearized',
    inputs=('δd', 'δPcpl'),
    outputs=('δiL', 'δvC'),
    states=('δiL', 'δvC')
)

### 6.3 Simulação do Sistema Linearizado

In [ ]:
# Obtenção da Saída do Sistema Não-Linear Trasladado
t, y_linear = ct.input_output_response(
    sys=buck_linearized, T=timepts,
    U=δU.tolist(),
    X0=δX0.tolist(),
)

# Criação e apresentação do gráfico
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico 1 - Corrente do Indutor
axs[0].plot(
    t,
    y_linear[0] + OP['iL'],
    linestyle='-', color='b', label='$\delta i_L$'
)

axs[0].set_xlabel('Tempo(s)')
axs[0].set_ylabel('Valores de $\delta i_L\,(A)$')
axs[0].set_title('Corrente do Indutor')
axs[0].legend()
axs[0].grid(True)

# Gráfico 2 - Tensão do Capacitor
axs[1].plot(
    t,
    y_linear[1] + OP['vC'],
    linestyle='-', color='r', label='$\delta v_C$'
)

axs[1].set_xlabel('Tempo(s)')
axs[1].set_ylabel('Valores de $\delta v_C\,(V)$')
axs[1].set_title('Tensão do Capacitor')
axs[1].legend()
axs[1].grid(True)

fig.suptitle('Sistema Dinâmico Linearizado', fontsize=16)

plt.tight_layout()
plt.show()

#### 7. Comparação dos Resultados Obtidos na Simulação

Nesta seção, exploramos visualmente o comportamento do sistema através de gráficos. A biblioteca Matplotlib é utilizada para criar visualizações que representam a resposta do sistema ao longo do tempo.

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

linewidth = 1.5

# Plotando o primeiro gráfico para a corrente do indutor
line1, = axs[0].plot(t, y_translated_nonlinear[0] + params['op']['iL'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)

line2, = axs[0].plot(t, y_linear[0] + params['op']['iL'], linestyle='-',
                     color='#8b0000', linewidth=linewidth, alpha=1)

axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('Corrente (A)', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].set_title('Corrente do Indutor $i_L(t)$', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Plotando o segundo gráfico para a tensão do capacitor

axs[1].plot(t, y_translated_nonlinear[1] + params['op']['vC'],
            linestyle='-', color='#120a8f', label='Não linear Transladado', linewidth=linewidth)

axs[1].plot(t, y_linear[1] + params['op']['vC'], linestyle='-',
            color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('Tensão (V)', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].set_title('Tensão do Capacitor $v_C(t)$', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Criando uma única legenda para ambos os subplots
fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
           fontsize=18, loc='upper center', bbox_to_anchor=(.5, 0.0), fancybox=False, shadow=False, ncol=2)

# Ajustando o layout para evitar sobreposição
plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/buck/sim1/op1/result.eps', format='eps', bbox_inches='tight')

## 4. Static and Dynamic ETMs

In this section, the dynamic and static ETM models proposed for use in this project are implemented. This model is based on a sufficient condition to allow the simultaneous design of the ETM and the controller, ensuring the asymptotic stability of the origin of the closed-loop system. Additionally, it addresses an optimization problem aiming to extend the intervals between events, with the objective of minimizing the number of events generated by the ETM.

### 4.1 Solving the Multiobjective Optimization Problem

In [ ]:
# Definição dos parâmetros do problema de otimização
A = cp.Parameter((2, 2),
                 value=np.array([[A11, A12], [A21, A22]]))
BU = cp.Parameter((2, 1), value=np.array([[B11], [B12]]))
I = cp.Parameter((2, 2), name='I', value=np.identity(2))

# Definição das variáveis do problema de otimização
Ξ_TIL = cp.Variable((2, 2), name='Ξ_TIL', PSD=True)
Ψ_TIL = cp.Variable((2, 2), name='Ψ_TIL', PSD=True)
X = cp.Variable((2, 2), name='X', PSD=True)
K_TIL = cp.Variable((1, 2), name='K_TIL')

_lambda = .5

# Definição do problema: objetivo e restrições
obj = cp.Minimize(cp.trace(_lambda * Ξ_TIL + (1 - _lambda) * Ψ_TIL))

M11 = A @ X + BU @ K_TIL + X @ A.T + K_TIL.T @ BU.T
M12 = BU @ K_TIL
M13 = X

M21 = K_TIL.T @ BU.T
M22 = -Ξ_TIL
M23 = np.zeros(shape=(2, 2))

M31 = X
M32 = np.zeros(shape=(2, 2))
M33 = -Ψ_TIL

M = cp.bmat([[M11, M12, M13],
             [M21, M22, M23],
             [M31, M32, M33]])

print(M.shape)
constraints = [M << 0]
constraints += [1e-9 * np.eye(2) <= Ξ_TIL]
constraints += [Ξ_TIL <= 1e9 * np.eye(2)]
constraints += [1e-9 * np.eye(2) <= Ψ_TIL]
constraints += [Ψ_TIL <= 1e9 * np.eye(2)]
prob = cp.Problem(obj, constraints)

# Resolução do problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)

K = None
Ξ = None
Ψ = None
P = None

# Apresentação dos Resultados
if prob.status not in ["infeasible", "unbounded"]:
  print("Valor ótimo: %s\n" % prob.value)
  for variable in prob.variables():
    if len(variable.shape) == 2:
      show_matrix(variable.name(), variable.value)
    else:
      print(variable.name(), '=', variable.value, '\n')

  X_INV = np.linalg.inv(X.value)
  Ξ = X_INV @ Ξ_TIL.value @ X_INV

  _K = K_TIL @ X_INV
  K = _K.value

  show_matrix('K', K)

  Ψ = np.linalg.inv(Ψ_TIL.value)
  P = X_INV

  show_matrix('P', P)
  show_matrix('Ξ', Ξ),
  show_matrix('Ψ', Ψ),
else:
  print('O problema não é factível')


### Modelo de ETM Dinâmico Proposto

In [ ]:
zero = 0
event_times_linear = [0.]


def get_gama(current_states, last_states_sent):
  error = last_states_sent - current_states
  return current_states.T @ Ψ @ current_states - error.T @ Ξ @ error


def etm_output(t, x, u, params):
  global zero, event_times_linear

  if t != etm_output.previous_time:
    etm_output.previous_time = t
    if etm_output.first_simulation and t == 0.:
      etm_output.first_simulation = False

  last_states_sent = u[0:2]
  current_states = u[2:4]

  Γ = get_gama(current_states, last_states_sent)
  trigger = Γ < 0

  if etm_output.first_simulation and trigger:
    event_times_linear.append(t)

  state_to_sent = (current_states if trigger or t == 0. else last_states_sent)
  return [state_to_sent[0], state_to_sent[1]]


etm_output.previous_time = 0
etm_output.first_simulation = True

ETM = ct.ss(
    None, etm_output,
    name='etm',
    inputs=('x1_hat', 'x2_hat', 'x1', 'x2'),
    outputs=('x1', 'x2'),
)


def zoh_output(t, x, u, params):
  if t != zoh_output.previous_time:
    zoh_output.last_states_sent = zoh_output.previous
    zoh_output.previous_time = t
  zoh_output.previous = u
  return zoh_output.last_states_sent


zoh_output.previous_time = 0
zoh_output.previous = []
zoh_output.last_states_sent = [0, 0]

ZOH = ct.ss(
    None, zoh_output,
    name='zoh',
    inputs=('x1', 'x2'),
    outputs=('x1_hat', 'x2_hat'),
)


def control_output(t, x, u, params):
  duty_cycle = K @ u
  return [duty_cycle]


CONTROL = ct.ss(
    None, control_output,
    name='control',
    inputs=('x1_hat', 'x2_hat'),
    outputs=('u'),
)

CLOSED_LOOP_BUCK_SYSTEM = ct.interconnect(
    (buck_linearized, ETM, ZOH, CONTROL),
    connections=(
        # Conexão entre a saída do controlador e a planta
        ('buck_linearized.δd', 'control.u'),

        # Conexão entre as saídas do ZOH e da planta ao ETM
        ('etm.x1_hat', 'zoh.x1_hat'),
        ('etm.x2_hat', 'zoh.x2_hat'),
        ('etm.x1', 'buck_linearized.δiL'),
        ('etm.x2', 'buck_linearized.δvC'),

        # Conexão da saída do ETM no ZOH
        ('zoh.x1', 'etm.x1'),
        ('zoh.x2', 'etm.x2'),

        # Conexão da saída do ZOH no controlador
        ('control.x1_hat', 'zoh.x1_hat'),
        ('control.x2_hat', 'zoh.x2_hat'),
    ),
    name='closed_loop_buck_system',
    inplist=('buck_linearized.δPcpl'),
    outlist=('buck_linearized.δiL',
             'buck_linearized.δvC',
             'buck_linearized.δd',
             ),
    output=('δiL', 'δvC', 'u')
)

print(CLOSED_LOOP_BUCK_SYSTEM)
print('')

δPcpl = δU[1]

t, y_linear = ct.input_output_response(
    sys=CLOSED_LOOP_BUCK_SYSTEM, T=timepts,
    U=δPcpl,
    X0=δX0,
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': step},
    params=params
)

inter_event_times_linear = [0.]

for i in range(1, len(event_times_linear)):
  inter_event_times_linear.append(
      event_times_linear[i] - event_times_linear[i-1])

In [ ]:
zero = 0
event_times_nonlinear = [0.]


def get_gama(current_states, last_states_sent):
  error = last_states_sent - current_states
  return current_states.T @ Ψ @ current_states - error.T @ Ξ @ error


def etm_output(t, x, u, params):
  global zero, event_times_nonlinear

  if t != etm_output.previous_time:
    etm_output.previous_time = t
    if etm_output.first_simulation and t == 0.:
      etm_output.first_simulation = False

  last_states_sent = u[0:2]
  current_states = u[2:4]

  Γ = get_gama(current_states, last_states_sent)
  trigger = Γ < 0

  if etm_output.first_simulation and trigger:
    event_times_nonlinear.append(t)

  state_to_sent = (current_states if trigger or t == 0. else last_states_sent)
  return [state_to_sent[0], state_to_sent[1]]


etm_output.previous_time = 0
etm_output.first_simulation = True

ETM = ct.ss(
    None, etm_output,
    name='etm',
    inputs=('x1_hat', 'x2_hat', 'x1', 'x2'),
    outputs=('x1', 'x2'),
)


def zoh_output(t, x, u, params):
  if t != zoh_output.previous_time:
    zoh_output.last_states_sent = zoh_output.previous
    zoh_output.previous_time = t
  zoh_output.previous = u
  return zoh_output.last_states_sent


zoh_output.previous_time = 0
zoh_output.previous = []
zoh_output.last_states_sent = [0, 0]

ZOH = ct.ss(
    None, zoh_output,
    name='zoh',
    inputs=('x1', 'x2'),
    outputs=('x1_hat', 'x2_hat'),
)


def control_output(t, x, u, params):
  duty_cycle = K @ u
  return [duty_cycle]


CONTROL = ct.ss(
    None, control_output,
    name='control',
    inputs=('x1_hat', 'x2_hat'),
    outputs=('u'),
)

CLOSED_LOOP_BUCK_SYSTEM = ct.interconnect(
    (translated_nonlinear_system, ETM, ZOH, CONTROL),
    connections=(
        # Conexão entre a saída do controlador e a planta
        ('translated_nonlinear_system.δd', 'control.u'),

        # Conexão entre as saídas do ZOH e da planta ao ETM
        ('etm.x1_hat', 'zoh.x1_hat'),
        ('etm.x2_hat', 'zoh.x2_hat'),
        ('etm.x1', 'translated_nonlinear_system.δiL'),
        ('etm.x2', 'translated_nonlinear_system.δvC'),

        # Conexão da saída do ETM no ZOH
        ('zoh.x1', 'etm.x1'),
        ('zoh.x2', 'etm.x2'),

        # Conexão da saída do ZOH no controlador
        ('control.x1_hat', 'zoh.x1_hat'),
        ('control.x2_hat', 'zoh.x2_hat'),
    ),
    name='closed_loop_buck_system',
    inplist=('translated_nonlinear_system.δPcpl'),
    outlist=('translated_nonlinear_system.δiL',
             'translated_nonlinear_system.δvC',
             'translated_nonlinear_system.δd',
             ),
    output=('δiL', 'δvC', 'u')
)

print(CLOSED_LOOP_BUCK_SYSTEM)
print('')

δPcpl = δU[1]

t, y_nonlinear = ct.input_output_response(
    sys=CLOSED_LOOP_BUCK_SYSTEM, T=timepts,
    U=δPcpl,
    X0=δX0,
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': step},
    params=params
)

inter_event_times_nonlinear = [0.]

for i in range(1, len(event_times_nonlinear)):
  inter_event_times_nonlinear.append(
      event_times_nonlinear[i] - event_times_nonlinear[i-1])

if len(event_times_nonlinear) == 1:
  inter_event_times_nonlinear.append(timepts[-1])

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 3))

linewidth = 1.5

# Plotando o primeiro gráfico para a corrente do indutor
line1, = axs[0].plot(t, y_nonlinear[0] + params['op']['iL'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)

line2, = axs[0].plot(t, y_linear[0] + params['op']['iL'], linestyle='-',
                     color='#8b0000', linewidth=linewidth, alpha=1)

axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('Corrente (A)', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].set_title('Corrente do Indutor $i_L(t)$', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Plotando o segundo gráfico para a tensão do capacitor

axs[1].plot(t, y_nonlinear[1] + params['op']['vC'],
            linestyle='-', color='#120a8f', label='Não linear Transladado', linewidth=linewidth)

axs[1].plot(t, y_linear[1] + params['op']['vC'], linestyle='-',
            color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('Tensão (V)', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].set_title('Tensão do Capacitor $v_C(t)$', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Criando uma única legenda para ambos os subplots
fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
           fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)

# Ajustando o layout para evitar sobreposição
plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/static-etm/buck/sim1/op1/result.eps', format='eps', bbox_inches='tight')

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 2.75))

linewidth = 2.

line1, = axs[0].plot(t[1:], y_nonlinear[2][1:] + params['op']['d'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)
axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('$d(t)$', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


line2, = axs[1].plot(t[1:], y_linear[2][1:] + params['op']['d'], linestyle='-',
                     color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('$d(t)$', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


# fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
#            fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)

# plt.suptitle(
#     'Duty Cycle $d(t)$', fontsize=18, fontweight='bold')

plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/static-etm/buck/sim1/op1/duty-cycle.eps', format='eps', bbox_inches='tight')

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 2.75))

linewidth = 2.

line1 = axs[0].stem(event_times_nonlinear, inter_event_times_nonlinear,
                    linefmt='#120a8f', markerfmt='o', basefmt=' ', bottom=0)
axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('ITEE (s)', fontsize=18)
axs[0].grid(linestyle='--')
# axs[0].set_title('Intervalo entre eventos', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


line2 = axs[1].stem(event_times_linear, inter_event_times_linear,
                    linefmt='#8b0000', markerfmt='o', basefmt=' ', bottom=0)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('ITEE (s)', fontsize=18)
axs[1].grid(linestyle='--')
# axs[1].set_title('Intervalo entre eventos', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
           fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)


plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/static-etm/buck/sim1/op1/inter-event-times.eps', format='eps', bbox_inches='tight')

In [ ]:
def tempo_acomodacao(sinal, tempo):
  valor_final = sinal[-1]  # Último valor do sinal
  limite = 0.02 * valor_final

  for i in range(len(sinal) - 1, -1, -1):
    if abs(sinal[i] - valor_final) >= limite:
      return tempo[i], i
  return tempo[0], 0


mean = np.mean(np.array(inter_event_times_nonlinear))
print(f'Média ITEE: %.3e' % mean)
mean = np.mean(np.array(inter_event_times_linear))
print(f'Média ITEE: %.3e' % mean)

ts_nonlinear_value, ts_nonlinear_idx = tempo_acomodacao(
    y_nonlinear[1] + OP['vC'], t)
print(f'Tempo de Acomodação (não linear): %.3e' % ts_nonlinear_value)
ts_linear_value, ts_linear_idx = tempo_acomodacao(y_linear[1] + OP['vC'], t)
print(f'Tempo de Acomodação (linear): %.3e' % ts_linear_value)

In [ ]:
ise = np.trapz(y_nonlinear[1][:ts_nonlinear_idx] ** 2, t[:ts_nonlinear_idx])
print('ise (não linear): %.2e' % ise)

ise = np.trapz(y_linear[1][:ts_linear_idx] ** 2, t[:ts_linear_idx])
print('ise (linear): %.2e' % ise)

itse = np.trapz(t[:ts_nonlinear_idx] * y_nonlinear[1]
                [:ts_nonlinear_idx] ** 2, t[:ts_nonlinear_idx])
print('itse (não linear): %.2e' % itse)

itse = np.trapz(t[:ts_linear_idx] * y_linear[1]
                [:ts_linear_idx] ** 2, t[:ts_linear_idx])

print('itse (linear): %.2e' % itse)

isc = np.trapz((y_nonlinear[2]
               [:ts_nonlinear_idx] + OP['d']) ** 2, t[:ts_nonlinear_idx])
print('isc (não linear): %.2e' % isc)

isc = np.trapz((y_linear[2]
               [:ts_linear_idx] + + OP['d']) ** 2, t[:ts_linear_idx])

print('isc (linear): %.2e' % isc)

# ETM Dinâmico

In [ ]:
zero = 0
event_times_linear = [0.]

θ = 1
λ = 100


def etm_update(t, n, u, params):
  last_states_sent = u[0:2]
  current_states = u[2:4]
  Γ = get_gama(current_states, last_states_sent)
  dn = -λ * n + Γ
  return [dn]


def etm_output(t, n, u, params):
  global zero, event_times_linear

  if t != etm_output.previous_time:
    etm_output.previous_time = t
    if etm_output.first_simulation and t == 0.:
      etm_output.first_simulation = False

  last_states_sent = u[0:2]
  current_states = u[2:4]

  Γ = get_gama(current_states, last_states_sent)
  trigger = (n + θ * Γ) < 0

  if etm_output.first_simulation and trigger:
    event_times_linear.append(t)

  state_to_sent = (current_states if trigger or t == 0. else last_states_sent)
  return [state_to_sent[0], state_to_sent[1], n[0]]


etm_output.previous_time = 0
etm_output.first_simulation = True

ETM = ct.NonlinearIOSystem(
    etm_update, etm_output,
    name='etm',
    states=('n'),
    inputs=('x1_hat', 'x2_hat', 'x1', 'x2'),
    outputs=('x1', 'x2', 'n'),
)


def zoh_output(t, x, u, params):
  if t != zoh_output.previous_time:
    zoh_output.lss = zoh_output.previous
    zoh_output.previous_time = t

  zoh_output.previous = u

  return zoh_output.lss


zoh_output.previous_time = 0
zoh_output.second_simulation = False
zoh_output.previous = []
zoh_output.lss = δX0.tolist()

ZOH = ct.ss(
    None, zoh_output,
    name='zoh',
    inputs=('x1', 'x2'),
    outputs=('x1_hat', 'x2_hat'),
    # states=1
)


def control_output(t, x, u, params):
  duty_cycle = K @ u
  return [duty_cycle]


CONTROL = ct.NonlinearIOSystem(
    None, control_output,
    name='control',
    inputs=('x1_hat', 'x2_hat'),
    outputs=('u'),
)

SYS = ct.interconnect(
    (buck_linearized, ETM, ZOH, CONTROL),
    connections=(
        # Conexão entre a saída do controlador e a planta
        ('buck_linearized.δd', 'control.u'),

        # Conexão entre as saídas do ZOH e da planta ao ETM
        ('etm.x1_hat', 'zoh.x1_hat'),
        ('etm.x2_hat', 'zoh.x2_hat'),
        ('etm.x1', 'buck_linearized.δiL'),
        ('etm.x2', 'buck_linearized.δvC'),

        # Conexão da saída do ETM no ZOH
        ('zoh.x1', 'etm.x1'),
        ('zoh.x2', 'etm.x2'),

        # Conexão da saída do ZOH no controlador
        ('control.x1_hat', 'zoh.x1_hat'),
        ('control.x2_hat', 'zoh.x2_hat'),
    ),
    name='conversor_system',
    inplist=('buck_linearized.δPcpl'),
    outlist=('buck_linearized.δiL',
             'buck_linearized.δvC',
             'buck_linearized.δd',
             'etm.n',
             ),
    output=('δIl', 'δVc', 'u', 'n')
)

print(SYS)
print('')

δPcpl = δU[1]

t, y_linear = ct.input_output_response(
    sys=SYS, T=timepts,
    U=δPcpl,
    X0=δX0 + [0.],
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': step}
)

inter_event_times_linear = [0.]

for i in range(1, len(event_times_linear)):
  inter_event_times_linear.append(
      event_times_linear[i] - event_times_linear[i-1])

In [ ]:
zero = 0
event_times_nonlinear = [0.]


def etm_update(t, n, u, params):
  last_states_sent = u[0:2]
  current_states = u[2:4]
  Γ = get_gama(current_states, last_states_sent)
  dn = -λ * n + Γ

  return [dn]


def etm_output(t, n, u, params):
  global zero, event_times_nonlinear

  if t != etm_output.previous_time:
    etm_output.previous_time = t
    if etm_output.first_simulation and t == 0.:
      etm_output.first_simulation = False

  last_states_sent = u[0:2]
  current_states = u[2:4]

  Γ = get_gama(current_states, last_states_sent)
  trigger = (n + θ * Γ) < 0

  if etm_output.first_simulation and trigger:
    event_times_nonlinear.append(t)

  state_to_sent = (current_states if trigger or t == 0. else last_states_sent)
  return [state_to_sent[0], state_to_sent[1], n[0]]


etm_output.previous_time = 0
etm_output.first_simulation = True

ETM = ct.NonlinearIOSystem(
    etm_update, etm_output,
    name='etm',
    states=('n'),
    inputs=('x1_hat', 'x2_hat', 'x1', 'x2'),
    outputs=('x1', 'x2', 'n'),
)


def zoh_output(t, x, u, params):
  if t != zoh_output.previous_time:
    zoh_output.lss = zoh_output.previous
    zoh_output.previous_time = t

  zoh_output.previous = u

  return zoh_output.lss


zoh_output.previous_time = 0
zoh_output.second_simulation = False
zoh_output.previous = []
zoh_output.lss = δX0.tolist()

ZOH = ct.ss(
    None, zoh_output,
    name='zoh',
    inputs=('x1', 'x2'),
    outputs=('x1_hat', 'x2_hat'),
    # states=1
)


def control_output(t, x, u, params):
  duty_cycle = K @ u
  return [duty_cycle]


CONTROL = ct.NonlinearIOSystem(
    None, control_output,
    name='control',
    inputs=('x1_hat', 'x2_hat'),
    outputs=('u'),
)

SYS = ct.interconnect(
    (translated_nonlinear_system, ETM, ZOH, CONTROL),
    connections=(
        # Conexão entre a saída do controlador e a planta
        ('translated_nonlinear_system.δd', 'control.u'),

        # Conexão entre as saídas do ZOH e da planta ao ETM
        ('etm.x1_hat', 'zoh.x1_hat'),
        ('etm.x2_hat', 'zoh.x2_hat'),
        ('etm.x1', 'translated_nonlinear_system.δiL'),
        ('etm.x2', 'translated_nonlinear_system.δvC'),

        # Conexão da saída do ETM no ZOH
        ('zoh.x1', 'etm.x1'),
        ('zoh.x2', 'etm.x2'),

        # Conexão da saída do ZOH no controlador
        ('control.x1_hat', 'zoh.x1_hat'),
        ('control.x2_hat', 'zoh.x2_hat'),
    ),
    name='conversor_system',
    inplist=('translated_nonlinear_system.δPcpl'),
    outlist=('translated_nonlinear_system.δiL',
             'translated_nonlinear_system.δvC',
             'translated_nonlinear_system.δd',
             'etm.n',
             ),
    output=('δIl', 'δVc', 'u', 'n')
)

print(SYS)
print('')

δPcpl = δU[1]

t, y_nonlinear = ct.input_output_response(
    sys=SYS, T=timepts,
    U=δPcpl,
    X0=δX0 + [0.],
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': step},
    params=params
)

inter_event_times_nonlinear = [0.]

for i in range(1, len(event_times_nonlinear)):
  inter_event_times_nonlinear.append(
      event_times_nonlinear[i] - event_times_nonlinear[i-1])

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 3))

linewidth = 1.5

# Plotando o primeiro gráfico para a corrente do indutor
line1, = axs[0].plot(t, y_nonlinear[0] + params['op']['iL'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)

line2, = axs[0].plot(t, y_linear[0] + params['op']['iL'], linestyle='-',
                     color='#8b0000', linewidth=linewidth, alpha=1)

axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('Corrente (A)', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].set_title('Corrente do Indutor $i_L(t)$', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Plotando o segundo gráfico para a tensão do capacitor

axs[1].plot(t, y_nonlinear[1] + params['op']['vC'],
            linestyle='-', color='#120a8f', label='Não linear Transladado', linewidth=linewidth)

axs[1].plot(t, y_linear[1] + params['op']['vC'], linestyle='-',
            color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('Tensão (V)', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].set_title('Tensão do Capacitor $v_C(t)$', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Criando uma única legenda para ambos os subplots
fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
           fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)

# Ajustando o layout para evitar sobreposição
plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/dynamic-etm/buck/sim1/op1/result.eps', format='eps', bbox_inches='tight')

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 2.75))

linewidth = 2.

line1, = axs[0].plot(t[1:], y_nonlinear[2][1:] + params['op']['d'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)
axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('$d(t)$', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


line2, = axs[1].plot(t[1:], y_linear[2][1:] + params['op']['d'], linestyle='-',
                     color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('$d(t)$', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


# fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
#            fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)

# plt.suptitle(
#     'Duty Cycle $d(t)$', fontsize=18, fontweight='bold')

plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/dynamic-etm/buck/sim1/op1/duty-cycle.eps', format='eps', bbox_inches='tight')

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 2.75))

linewidth = 2.

line1 = axs[0].stem(event_times_nonlinear, inter_event_times_nonlinear,
                    linefmt='#120a8f', markerfmt='o', basefmt=' ', bottom=0)
axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('ITEE (s)', fontsize=18)
axs[0].grid(linestyle='--')
# axs[0].set_title('Intervalo entre eventos', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


line2 = axs[1].stem(event_times_linear, inter_event_times_linear,
                    linefmt='#8b0000', markerfmt='o', basefmt=' ', bottom=0)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('ITEE (s)', fontsize=18)
axs[1].grid(linestyle='--')
# axs[1].set_title('Intervalo entre eventos', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
#            fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)


plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/dynamic-etm/buck/sim1/op1/inter-event-times.eps', format='eps', bbox_inches='tight')

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 3))

linewidth = 2.

line1, = axs[0].plot(t, y_nonlinear[3],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)
axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('$\eta(t)$', fontsize=18)
axs[0].grid(linestyle='--')
# axs[0].set_title('$\eta(t)$  $d(t)$', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)
# axs[0].ticklabel_format(style='plain')


line2, = axs[1].plot(t, y_linear[3], linestyle='-',
                     color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('$\eta(t)$', fontsize=18)
axs[1].grid(linestyle='--')
# axs[1].set_title('Duty Cycle  $d(t)$', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)
# axs[1].ticklabel_format(style='plain')


fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
           fontsize=18, loc='upper center', bbox_to_anchor=(.5, 0.0), fancybox=False, shadow=False, ncol=2)

plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/dynamic-etm/buck/sim1/op1/eta.eps', format='eps', bbox_inches='tight')

In [ ]:
def tempo_acomodacao(sinal, tempo, percentagem=2):
  valor_final = sinal[-1]  # Último valor do sinal
  limite = valor_final * (percentagem / 100)

  for i in range(len(sinal) - 1, -1, -1):
    if abs(sinal[i] - valor_final) >= limite:
      return tempo[i], i
  return tempo[0], 0


mean = np.mean(np.array(inter_event_times_nonlinear))
print(f'Média ITEE: %.3e' % mean)
mean = np.mean(np.array(inter_event_times_linear))
print(f'Média ITEE: %.3e' % mean)

ts_nonlinear_value, ts_nonlinear_idx = tempo_acomodacao(y_nonlinear[1], t)
print(f'Tempo de Acomodação (não linear): %.3e' % ts_nonlinear_value)
ts_linear_value, ts_linear_idx = tempo_acomodacao(y_linear[1], t)
print(f'Tempo de Acomodação (linear): %.3e' % ts_nonlinear_value)

In [ ]:
ise = np.trapz(y_nonlinear[1][:ts_nonlinear_idx] ** 2, t[:ts_nonlinear_idx])
print('ise (não linear): %.2e' % ise)

ise = np.trapz(y_linear[1][:ts_linear_idx] ** 2, t[:ts_linear_idx])
print('ise (linear): %.2e' % ise)

itse = np.trapz(t[:ts_nonlinear_idx] * y_nonlinear[1]
                [:ts_nonlinear_idx] ** 2, t[:ts_nonlinear_idx])
print('itse (não linear): %.2e' % itse)

itse = np.trapz(t[:ts_linear_idx] * y_linear[1]
                [:ts_linear_idx] ** 2, t[:ts_linear_idx])

print('itse (linear): %.2e' % itse)

isc = np.trapz((y_nonlinear[2]
               [:ts_nonlinear_idx] + OP['d']) ** 2, t[:ts_nonlinear_idx])
print('isc (não linear): %.2e' % isc)

isc = np.trapz((y_linear[2]
               [:ts_linear_idx] + + OP['d']) ** 2, t[:ts_linear_idx])

print('isc (linear): %.2e' % isc)

In [ ]:
# Definição dos parâmetros do problema de otimização
A = cp.Parameter((2, 2),
                 value=np.array([[A11, A12], [A21, A22]]))
BU = cp.Parameter((2, 1), value=np.array([[B11], [B12]]))

# Definição das variáveis do problema de otimização
P = cp.Variable((2, 2), name='P', PSD=True)
W = cp.Variable((1, 2), name='W')


# Definição do problema: objetivo e restrições
obj = cp.Minimize(0)
constraints = [A @ P + P @ A.T + BU @ W + W.T @ BU.T << 0]
prob = cp.Problem(obj, constraints)

# Resolução do problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)

K = None

# Apresentação dos Resultados
if prob.status not in ["infeasible", "unbounded"]:
  print("Valor ótimo: %s\n" % prob.value)
  for variable in prob.variables():
    if len(variable.shape) == 2:
      show_matrix(variable.name(), variable.value)
    else:
      print(variable.name(), '=', variable.value, '\n')

  P_INV = np.linalg.inv(P.value)

  _K = W @ P_INV
  K = _K.value

  show_matrix('K', K)
else:
  print('O problema não é factível')

In [ ]:
def control_output(t, x, u, params):
  duty_cycle = K @ u
  return [duty_cycle]


CONTROL = ct.ss(
    None, control_output,
    name='control',
    inputs=('x1', 'x2'),
    outputs=('u'),
)

CLOSED_LOOP_BUCK_SYSTEM = ct.interconnect(
    (buck_linearized, CONTROL),
    connections=(
        # Conexão entre a saída do controlador e a planta
        ('buck_linearized.δd', 'control.u'),

        # Conexão da saída do ZOH no controlador
        ('control.x1', 'buck_linearized.δiL'),
        ('control.x2', 'buck_linearized.δvC'),
    ),
    name='closed_loop_buck_system',
    inplist=('buck_linearized.δPcpl'),
    outlist=('buck_linearized.δiL',
             'buck_linearized.δvC',
             'buck_linearized.δd',
             ),
    output=('δiL', 'δvC', 'u')
)

δPcpl = δU[1]

t, y_linear = ct.input_output_response(
    sys=CLOSED_LOOP_BUCK_SYSTEM, T=timepts,
    U=δPcpl,
    X0=δX0,
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': step},
    params=params
)

In [ ]:
def control_output(t, x, u, params):
  duty_cycle = K @ u
  return [duty_cycle]


CONTROL = ct.ss(
    None, control_output,
    name='control',
    inputs=('x1', 'x2'),
    outputs=('u'),
)

CLOSED_LOOP_BUCK_SYSTEM = ct.interconnect(
    (translated_nonlinear_system, CONTROL),
    connections=(
        # Conexão entre a saída do controlador e a planta
        ('translated_nonlinear_system.δd', 'control.u'),

        # Conexão da saída do ZOH no controlador
        ('control.x1', 'translated_nonlinear_system.δiL'),
        ('control.x2', 'translated_nonlinear_system.δvC'),
    ),
    name='closed_loop_buck_system',
    inplist=('translated_nonlinear_system.δPcpl'),
    outlist=('translated_nonlinear_system.δiL',
             'translated_nonlinear_system.δvC',
             'translated_nonlinear_system.δd',
             ),
    output=('δiL', 'δvC', 'u')
)

δPcpl = δU[1]

t, y_nonlinear = ct.input_output_response(
    sys=CLOSED_LOOP_BUCK_SYSTEM, T=timepts,
    U=δPcpl,
    X0=δX0,
    solve_ivp_method='RK45',
    solve_ivp_kwargs={'max_step': step},
    params=params
)

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 3))

linewidth = 1.5

# Plotando o primeiro gráfico para a corrente do indutor
line1, = axs[0].plot(t, y_nonlinear[0] + params['op']['iL'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)

line2, = axs[0].plot(t, y_linear[0] + params['op']['iL'], linestyle='-',
                     color='#8b0000', linewidth=linewidth, alpha=1)

axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('Corrente (A)', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].set_title('Corrente do Indutor $i_L(t)$', fontsize=20)
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Plotando o segundo gráfico para a tensão do capacitor

axs[1].plot(t, y_nonlinear[1] + params['op']['vC'],
            linestyle='-', color='#120a8f', label='Não linear Transladado', linewidth=linewidth)

axs[1].plot(t, y_linear[1] + params['op']['vC'], linestyle='-',
            color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('Tensão (V)', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].set_title('Tensão do Capacitor $v_C(t)$', fontsize=20)
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)

# Criando uma única legenda para ambos os subplots
fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
           fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)

# Ajustando o layout para evitar sobreposição
plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/classic/buck/sim1/op1/result.eps', format='eps', bbox_inches='tight')

In [ ]:
# Definindo o tamanho da figura e os subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 2.75))

linewidth = 2.

line1, = axs[0].plot(t[1:], y_nonlinear[2][1:] + params['op']['d'],
                     linestyle='-', color='#120a8f', linewidth=linewidth, alpha=1)
axs[0].set_xlabel('Tempo (s)', fontsize=18)
axs[0].set_ylabel('$d(t)$', fontsize=18)
axs[0].grid(linestyle='--')
axs[0].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


line2, = axs[1].plot(t[1:], y_linear[2][1:] + params['op']['d'], linestyle='-',
                     color='#8b0000', label='Linearizado', linewidth=linewidth)

axs[1].set_xlabel('Tempo (s)', fontsize=18)
axs[1].set_ylabel('$d(t)$', fontsize=18)
axs[1].grid(linestyle='--')
axs[1].tick_params(axis='both', direction='in', length=4, width=1,
                   colors='black', top=True, right=True, labelsize=16)


# fig.legend([line1, line2], ['Não Linear', 'Linearizado'],
#            fontsize=14, loc='upper center', bbox_to_anchor=(.5, 1.2), fancybox=False, shadow=False, ncol=2)

# plt.suptitle(
#     'Duty Cycle $d(t)$', fontsize=18, fontweight='bold')

plt.tight_layout()

plt.savefig(
    '/home/andel/Projetos/Latex/[TCC] Andevaldo Vitório/figuras/classic/buck/sim1/op1/duty-cycle.eps', format='eps', bbox_inches='tight')